# Importing the Data

**Goal**: Classify attacks assuming a multiclass classification system has alerted an attack.

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 

from sklearn.metrics import classification_report, accuracy_score

In [0]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Research/'  #change dir to your project folder

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#cleaned data without non-attack values
X = pd.read_csv(root_path+'Data/Features.csv', index_col=0)
Y = pd.read_csv(root_path+'Data/Target.csv', index_col=0)

# Feature Engineering

In [0]:
# %%capture warnings1
# cluster and score
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.cluster import completeness_score

score = []
for i in range(len(X.columns)): # loop number of features
  K = MiniBatchKMeans(n_clusters=len(Y.attack_cat.unique())-1, random_state=0)
  pred = K.fit_predict(X.iloc[:, [i]].values)
  s = completeness_score(Y[Y.columns[0]].values,pred)
  score.append(s)
  


In [0]:
# Rank the features and sort

s2 = score
np.asarray(s2)

s1 = []
for i in range(len(X.columns)):
  s1.append(i)
  
np.asarray(s1)

li = list(zip(s1, s2))

def sortSecond(val): 
    return val[1] 
  
li.sort(key = sortSecond, reverse=True) 


print("Index of Feature , adjusted_rand_score\n")
for i in range(len(X.columns)):
    print(li[i])


Index of Feature , adjusted_rand_score

(31, 1.0)
(34, 1.0)
(35, 1.0)
(9, 0.6176442600463541)
(3, 0.5213330140825623)
(1, 0.459501975281486)
(14, 0.4520491087370686)
(15, 0.4520319903543966)
(6, 0.44199368681334195)
(32, 0.44074102877494076)
(5, 0.42682927067764354)
(18, 0.40644319057101774)
(4, 0.38553696493589973)
(19, 0.37146556822087723)
(13, 0.3688367880115802)
(12, 0.33836485488716744)
(7, 0.33520809711462446)
(2, 0.3104553828145945)
(23, 0.30837430741134275)
(27, 0.2971901012828065)
(8, 0.29517186515824734)
(22, 0.2942620218567771)
(0, 0.2780497434395166)
(28, 0.27725939767775915)
(26, 0.2769064817114117)
(30, 0.2741146261881654)
(29, 0.261110495703687)
(40, 0.252392036948846)
(17, 0.2521422441276322)
(16, 0.24948368741117075)
(33, 0.24841727748592937)
(20, 0.24839062384568847)
(41, 0.24194847310617873)
(37, 0.2361213155704215)
(36, 0.2296657332975711)
(11, 0.2270803889323787)
(38, 0.22656387392369598)
(42, 0.2250941558313391)
(21, 0.21053943581560194)
(10, 0.19970591627256387)


In [0]:
# Create a copy of X dataframe with columns sorted by score

titles = []

for i in range(len(X.columns)):
  p = X.columns[li[i][0]]
  titles.append(p)


X1 = pd.DataFrame(columns=titles)

for i in range(len(X.columns)):
  X1[X1.columns[i]] = X[X.columns[li[i][0]]]




In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import accuracy_score

In [0]:
# RF w/out recursive Feature Elimination
x_train, x_test, y_train, y_test = train_test_split(X1, Y, random_state=11)
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred, digits=4))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


                precision    recall  f1-score   support

      Analysis     0.6750    0.1745    0.2773       619
      Backdoor     0.5976    0.0864    0.1510       567
           DoS     0.3181    0.2712    0.2928      4075
      Exploits     0.6353    0.8086    0.7115     11236
       Fuzzers     0.8978    0.8647    0.8809      6127
       Generic     0.9967    0.9858    0.9912     53886
Reconnaissance     0.9205    0.7459    0.8241      3385
     Shellcode     0.6704    0.6134    0.6406       388
         Worms     1.0000    0.2895    0.4490        38

      accuracy                         0.8907     80321
     macro avg     0.7457    0.5378    0.5798     80321
  weighted avg     0.8941    0.8907    0.8878     80321



In [0]:
# %%capture warnings
# Recursive Feature Elemination from # of features to 0 and keep the accuracy score of each

accuracy = []
X2 = X1.copy()

# for i in range(len(X1.columns)-1,-1,-1):
for i in range(len(X1.columns)-1): 
  x_train, x_test, y_train, y_test = train_test_split(X1, Y, test_size=0.2, random_state=11)
  X1.drop(X1.columns[len(X1.columns)-1], axis=1, inplace=True)
  clf = rf(random_state=0, n_jobs=-1)
  clf.fit(x_train, y_train)
  y_pred = clf.predict(x_test)
  accuracy.append(accuracy_score(y_test, y_pred)) 

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff

In [0]:
index = accuracy.index(max(accuracy))
# for i in accuracy:
#   print(i)
  

# print(accuracy)

print(max(accuracy), "max accuracy")

print(index, "index")


X3 = X2.iloc[:,0:len(X.columns)-index]

0.9018005820377546 max accuracy
26 index


# Performance Analysis

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X3, Y, test_size=0.2, random_state=11)

In [0]:
from sklearn.ensemble import RandomForestClassifier as rf
clf = rf(random_state=0, n_jobs=-1)
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
                       oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
x = list(zip(X3.columns, clf.feature_importances_))

def sortSecond(val): 
    return val[1] 
  
x.sort(key = sortSecond, reverse=True) 

for i in range(len(X3.columns)):
    print(x[i])

('sbytes', 0.3553773744110834)
('service', 0.3058906475821295)
('smeansz', 0.08228929775361374)
('dbytes', 0.06376128826472376)
('Spkts', 0.04960092986635579)
('Dpkts', 0.04262368380598087)
('dmeansz', 0.0342959501081554)
('state', 0.030920949492777038)
('sloss', 0.01454121356842657)
('sttl', 0.013885728433295191)
('dttl', 0.0037282209192233255)
('swin', 0.002267642590016797)
('ct_state_ttl', 0.0005807460563001704)
('dwin', 0.00010421882349157933)
('is_ftp_login', 7.421081727621609e-05)
('ct_ftp_cmd', 5.789750715064165e-05)
('is_sm_ips_ports', 0.0)


In [0]:
clf.score(x_train, y_train) 

0.9138063853462296

In [0]:
print("Accuracy")
clf.score(x_test, y_test) 

Accuracy


0.9018005820377546

In [0]:
y_pred = clf.predict(x_test)

print("Classification\n")
print(classification_report(y_test,y_pred, digits=4))

Classification

                precision    recall  f1-score   support

      Analysis     0.5748    0.1475    0.2347       495
      Backdoor     0.8125    0.1109    0.1951       469
           DoS     0.5948    0.1202    0.2001      3235
      Exploits     0.6155    0.9248    0.7391      8944
       Fuzzers     0.9137    0.8415    0.8761      4846
       Generic     0.9972    0.9898    0.9935     43219
Reconnaissance     0.9208    0.7634    0.8347      2696
     Shellcode     0.6408    0.6947    0.6667       321
         Worms     0.7742    0.7500    0.7619        32

      accuracy                         0.9018     64257
     macro avg     0.7605    0.5936    0.6113     64257
  weighted avg     0.9078    0.9018    0.8892     64257

